# Test №1

In [1]:
# делаем import необходимых библиотек
import os
import mlflow

# устанавливаем локальное хранилище для наших экспериментов
# хранилище должно быть такое же, как и при запуске сервиса
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# получаем id эксеримента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

# залогируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 123)
    mlflow.log_artifact("test_artifact.txt", artifact_path="test_artifact")

print(f"Run id запуска: {run_id}")

Run id запуска: 17aae0ab1c7a474684ae4ea831002ab5


In [2]:
!ls mlflow_experiments_store/0/{run_id}/metrics

test_metric


In [3]:
!cat mlflow_experiments_store/0/{run_id}/metrics/test_metric.txt

cat: mlflow_experiments_store/0/17aae0ab1c7a474684ae4ea831002ab5/metrics/test_metric.txt: No such file or directory


In [4]:
!ls mlflow_experiments_store/0/{run_id}/artifacts/test_artifact

test_artifact.txt


In [5]:
!cat mlflow_experiments_store/0/{run_id}/artifacts/test_artifact/test_artifact.txt

test_artifact_123

# Test №2

In [15]:
assert os.path.exists("mlflow_experiments_store_sqlite")
assert os.path.exists("mydb.sqlite") 

In [21]:
import os
import mlflow

mlflow.set_tracking_uri('http://0.0.0.0:5000')

# получаем id эксперимента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

with mlflow.start_run(run_name="Default", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric_sqlite", 123)
    mlflow.log_artifact("test_artifact.txt", artifact_path="test_artifact_sqlite")

# проверим, что наши данные сохранились в локальной папке, а также создалась база данных SQLite
assert os.path.exists("mlflow_experiments_store_sqlite")
assert os.path.exists("mydb.sqlite")

In [17]:
!ls mlflow_experiments_store_sqlite/0/{run_id}/artifacts/test_artifact_sqlite

test_artifact.txt


In [18]:
!cat mlflow_experiments_store_sqlite/0/{run_id}/artifacts/test_artifact_sqlite/test_artifact.txt

test_artifact_123

# Test №3

In [ ]:
import os
import mlflow

##from dotenv import load_dotenv
##load_dotenv()
# определяем основные credentials, которые нужны для подключения к MLflow
# важно, что credentials мы передаём для себя как пользователей Tracking Service
# у вас должен быть доступ к бакету, в который вы будете складывать артефакты
##os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.getenv("MLFLOW_S3_ENDPOINT_URL") #endpoint бакета от YandexCloud
##os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
##os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

# определяем глобальные переменные
# поднимаем MLflow локально
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

YOUR_NAME = "" # введите своё имя для создания уникального эксперимента
assert YOUR_NAME, "введите своё имя в переменной YOUR_NAME для создания уникального эксперимента"

# название тестового эксперимента и запуска (run) внутри него
EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = "test_connection_run"

# тестовые данные
METRIC_NAME = "test_metric"
METRIC_VALUE = 0

# устанавливаем host, который будет отслеживать наши эксперименты
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

# создаём тестовый эксперимент и записываем в него тестовую информацию
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    
    mlflow.log_metric(METRIC_NAME, METRIC_VALUE)

In [ ]:
##### 4. Проверяем себя, что в MLflow:
# - создался `experiment` с нашим именем
# - внутри эксперимента появился запуск `run`
# - внутри `run` записалась наша тестовая `metric`
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME]